<a href="https://colab.research.google.com/github/rahul-727/LLM-Lab-works/blob/main/Rahul_544_Lab1_PLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement transfer learning to fine-tune a language model on a Question Answering/Machine Reading Comprehension dataset.

In [8]:
!pip install datasets

In [9]:
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad", split={'train': 'train[:10%]', 'validation': 'validation[:10%]'})
print(dataset['train'][0])
print(dataset['validation'][0])

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}
{'id': '56be4db0acb8001400a502ec', 'title': 'Super_Bow

Loading a Pre-Trained Model
 For QA tasks, models like bert-base-uncased are commonly used.

In [10]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model_name = "bert-base-uncased"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preparing Data for Fine-Tuning

In [11]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=128,
        stride=32,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        answer_start = answers["answer_start"][0]
        answer_end = answer_start + len(answers["text"][0])

        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1

        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        if not (offsets[token_start_index][0] <= answer_start and offsets[token_end_index][1] >= answer_end):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= answer_start:
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)

            while offsets[token_end_index][1] >= answer_end:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

train_dataset = dataset["train"].map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)
validation_dataset = dataset["validation"].map(prepare_train_features, batched=True, remove_columns=dataset["validation"].column_names)


Map:   0%|          | 0/8760 [00:00<?, ? examples/s]

Map:   0%|          | 0/1057 [00:00<?, ? examples/s]

In [13]:
!pip install accelerate -U

Training the Model

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    save_steps=500,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

trainer.train()


Step,Training Loss,Validation Loss
500,1.569600,1.179439
1000,1.048400,0.987830
1500,0.643500,1.035177
2000,0.561200,0.987149
2500,0.378100,1.228360
3000,0.263600,1.244560


TrainOutput(global_step=3291, training_loss=0.6993942237559585, metrics={'train_runtime': 1330.6845, 'train_samples_per_second': 39.546, 'train_steps_per_second': 2.473, 'total_flos': 3437554807429632.0, 'train_loss': 0.6993942237559585, 'epoch': 3.0})

Manual Evaluation and Testing

In [16]:
import torch

def ask_question(question, context, model, tokenizer):
    device = next(model.parameters()).device

    inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    return answer

context = "The quick brown fox jumps over the lazy dog."
question = "Who jumps over the lazy dog?"
model = model.to('cuda')
answer = ask_question(question, context, model, tokenizer)
print(answer)


quick brown fox
